# STAC
Let's use EOReader to create [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) items.

<div class="alert alert-info">
  
   <strong>Note:</strong> This is experimental for now, use it at your own risk !
    
</div>

<div class="alert alert-warning">
  
<strong>Warning:</strong>
You will need to install <a href="https://pystac.readthedocs.io/en/stable/"><code>pystac[validation]</code></a>,
<a href="https://python-visualization.github.io/folium/"><code>folium</code></a>
and <a href="https://eodag.readthedocs.io/en/stable/"><code>eodag</code></a> (version != 2.6.0) to run this notebook
    
</div>

## Imports

In [1]:
# Imports
import os

import pystac
import geopandas as gpd
from tempfile import TemporaryDirectory
from shapely.geometry import mapping

from eodag import setup_logging
from eodag.api.core import EODataAccessGateway

from eoreader.reader import Reader

## Create logger

In [2]:
# Create logger
import logging
from sertit import logs

logger = logging.getLogger("eoreader")
logs.init_logger(logger)

## Linking some data 

Let's take 3 products covering approximately the same area (over DAX city in France):
- One Landsat-8 OLI-TIRS collection 2
- One Landsat-5 TM collection 2
- One Sentinel-2 L1C

In [3]:
prod_folder = os.path.join("/home", "prods")
paths = [
    # Landsat-8 OLI-TIRS collection 2
    os.path.join(prod_folder, "LANDSATS_COL2", "LC08_L1TP_200030_20201220_20210310_02_T1.tar"),
    # Landsat-5 TM collection 2    
    os.path.join(prod_folder, "LANDSATS_COL2", "LT05_L1TP_200030_20111110_20200820_02_T1.tar"),
    # Sentinel-2 L2A
    os.path.join(prod_folder, "S2", "PB 02.07+", "S2A_MSIL1C_20191215T110441_N0208_R094_T30TXP_20191215T114155.SAFE"),
]

## Create STAC catalog
Create a STAC catalog and add 3 STAC items to it.

In [4]:
# Create the reader
reader = Reader()

# Work in a temporary directory
tmp = TemporaryDirectory()

In [5]:
# Create STAC catalog
catalog_path = os.path.join(tmp.name, "catalog.json")
catalog = pystac.Catalog(
    id='SERTIT_101',
    description="SERTIT's Catalog",
    title='SERTIT Catalog',
    href=catalog_path
)

In [6]:
# Add all the products into the STAC catalog
for path in paths:
    logger.info(f"*** {os.path.basename(path)} ***")

    # Open the product
    prod = reader.open(path, remove_tmp=True)

    # Get item
    item = prod.stac.create_item()

    # Add item to catalogue
    catalog.add_item(item)

2023-05-31 11:47:09,915 - [INFO] - *** LC08_L1TP_200030_20201220_20210310_02_T1.tar ***
2023-05-31 11:47:34,019 - [WARNING] - No quicklook found in 20201220T104856_L8_200030_OLI_TIRS
2023-05-31 11:47:37,448 - [INFO] - *** LT05_L1TP_200030_20111110_20200820_02_T1.tar ***
2023-05-31 11:47:45,405 - [INFO] - *** S2A_MSIL1C_20191215T110441_N0208_R094_T30TXP_20191215T114155.SAFE ***


In [7]:
# Save catalog
catalog.describe()
catalog.normalize_and_save(tmp.name, catalog_type=pystac.CatalogType.SELF_CONTAINED)

* <Catalog id=SERTIT_101>
  * <Item id=20201220T104856_L8_200030_OLI_TIRS>
  * <Item id=20111110T103612_L5_200030_TM>
  * <Item id=20191215T110441_S2_T30TXP_L1C_114155>


In [8]:
list(catalog.get_items())[0]

id: 20201220T104856_L8_200030_OLI_TIRS
"bbox: [-2.744857724803896, 42.09665285173287, 0.14785265624107427, 44.2467651307631]"
tilename: 200030
eo:cloud_cover: 16.36
proj:epsg: 32630
"proj:wkt2: PROJCS[""WGS 84 / UTM zone 30N"",GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS 84"",6378137,298.257223563,AUTHORITY[""EPSG"",""7030""]],AUTHORITY[""EPSG"",""6326""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AUTHORITY[""EPSG"",""4326""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-3],PARAMETER[""scale_factor"",0.9996],PARAMETER[""false_easting"",500000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""32630""]]"
"proj:geometry: {'type': 'Polygon', 'coordinates': (((704535.0, 4665915.0), (703755.0, 4666095.0), (690465.0, 4669185.0), (630855.0, 4683045.0), (526215.0, 4707375.0), (523635.0, 4707975.0), (523125.0, 4708095.0), (523005.0, 4708125.0), (522945.0, 4708155.0), (522945.0, 4708275.0), (522975.0, 4708425.0), (523185.0, 4709355.0), (566355.0, 4898205.0), (566445.0, 4898595.0), (566475.0, 4898715.0), (566505.0, 4898715.0), (567045.0, 4898595.0), (567975.0, 4898385.0), (573405.0, 4897155.0), (586515.0, 4894185.0), (697485.0, 4869045.0), (748335.0, 4857525.0), (748575.0, 4857465.0), (748575.0, 4857375.0), (748455.0, 4856835.0), (747285.0, 4851675.0), (747135.0, 4851015.0), (745305.0, 4842975.0), (744675.0, 4840215.0), (742125.0, 4829055.0), (741315.0, 4825515.0), (739995.0, 4819755.0), (738825.0, 4814655.0), (729945.0, 4775955.0), (705225.0, 4668735.0), (704685.0, 4666395.0), (704565.0, 4665915.0), (704535.0, 4665915.0)),)}"
"proj:bbox: [521085.0, 4664985.0, 751515.0, 4899315.0]"
"proj:centroid: {'lat': 43.183607660455536, 'lon': -1.3295876133318103}"
"proj:shape: [7811, 7681]"
"proj:transform: [30.0, 0.0, 521085.0, 0.0, -30.0, 4899315.0, 0.0, 0.0, 1.0]"


# Query the catalog

`EODAG` is an opensource python library that implements STAC and allows you to query your local STAC catalog.  
Look at [here](https://eodag.readthedocs.io/en/stable/notebooks/tutos/tuto_stac_client.html) for a detailed tutorial.

In [9]:
# Create an EODAG custom STAC provider
dag = EODataAccessGateway()

# Set EODAG logging level to WARNING
setup_logging(verbose=1)

# Add the custom STAC provider, exactly like in the tutorial mentioned above
dag.update_providers_config("""
stac_http_provider:
    search:
        type: StaticStacSearch
        api_endpoint: %s
    products:
        GENERIC_PRODUCT_TYPE:
            productType: '{productType}'
    download:
        type: HTTPDownload
        base_uri: %s
        flatten_top_dirs: True
        outputs_prefix: %s
""" % (catalog_path, tmp.name, tmp.name))

# Set the custom STAC provider as preferred
dag.set_preferred_provider("stac_http_provider")

In [10]:
# Query every product from inside the catalog
all_products, _ = dag.search()

UnsupportedProvider: 

In [ ]:
# Load an AOI
aoi_path = os.path.join("/home", "aois", "DAX.geojson")
aoi = gpd.read_file(aoi_path)
aoi_geojson = mapping(aoi.geometry.values[0])

# Query spatially with the AOI and temporally with a time period
query_args = {"start": "2020-05-01", "end": "2022-05-06", "geom": aoi.geometry.values[0]}
query_products, _ = dag.search(**query_args)

In [ ]:
query_products[0]

In [ ]:
query_products[0].assets['Blue']

## Display the results

We can use `folium` to display the results geometry over a map.

In [ ]:
import folium

# Create a map zoomed over the search area
fmap = folium.Map((43.2, -1.05), zoom_start=7)

# Add a layer green layer for the query over the AOI
folium.GeoJson(
    data=all_products.as_geojson_object(),
    tooltip = "All products stored in the catalog",
    style_function=lambda x: {'color': 'green'}
).add_to(fmap)

# Add a layer green layer for the query over the AOI
folium.GeoJson(
    data=query_products.as_geojson_object(),
    tooltip = "Retrieved products with the query",
    style_function=lambda x: {'color': 'red'}
).add_to(fmap)

# Add a layer blue layer for the AOI
folium.GeoJson(
    data=aoi_geojson,
    tooltip = "DAX AOI",
    style_function=lambda x: {'color': 'blue'}
).add_to(fmap)

fmap

In [ ]:
# Clean the tmp directory
tmp.cleanup()
